In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter   
holiday - whether the day is considered a holiday  
workingday - whether the day is neither a weekend nor holiday  
weather -   
    1: Clear, Few clouds, Partly cloudy, Partly cloudy   
    2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist   
    3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds   
    4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog    
temp - temperature in Celsius   
atemp - "feels like" temperature in Celsius   
humidity - relative humidity  
windspeed - wind speed  
casual - number of non-registered user rentals initiated  
registered - number of registered user rentals initiated  
count - number of total rentals  

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
train.info()

In [ ]:
train.workingday.value_counts()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum() ## null 은 없음 

In [ ]:
train.workingday.count()

In [ ]:
train['datetime'] = train['datetime'].astype('datetime64[ns]') 

In [ ]:
test['datetime'] = test['datetime'].astype('datetime64[ns]') 

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour

In [ ]:
train.head()

In [ ]:
## year, month, day, hour
figure, ((ax1, ax2),(ax3,ax4)) = plt.subplots(nrows = 2, ncols = 2)
figure.set_size_inches(10,10)

sns.barplot(data=train, x='year', y='count', ax=ax1)
sns.barplot(data=train, x='month', y='count', ax=ax2)
sns.barplot(data=train, x='day', y='count', ax=ax3)
sns.barplot(data=train, x='hour', y='count', ax=ax4)

ax1.set(ylabel = 'Year', title = 'rent count / year')
ax2.set(ylabel = 'Month', title = 'rent count / month')
ax3.set(ylabel = 'day', title = 'rent count / day')
ax4.set(ylabel = 'hour', title = 'rent count / hour')

In [ ]:
# holiday - whether the day is considered a holiday
# workingday - whether the day is neither a weekend nor holiday
print(len(train[(train.holiday==0) & (train.workingday==1)]) / len(train) * 100 , "%") ## 평일
print(len(train[(train.holiday==0) & (train.workingday==0)]) / len(train) * 100 , "%") ## 주말
print(len(train[(train.holiday==1) & (train.workingday==0)]) / len(train) * 100 , "%") ## 공휴일
print(len(train[(train.holiday==0) & (train.workingday==0)]) / len(train) * 100 + len(train[(train.holiday==1) & (train.workingday==0)]) / len(train) * 100 , "%") ## 공휴일
print(len(train[(train.holiday==1) & (train.workingday==1)]) / len(train) * 100 , "%") ## X

## -> 평일에 타는 날이 제일 많다
## holiday 일자가 적기때문에 의미가 있는지는....

In [ ]:
labels = ['holiday', 'weekend', 'workingday']
sizes = [len(train[(train.holiday==1) & (train.workingday==0)]), len(train[(train.holiday==0) & (train.workingday==0)]) / len(train) * 100, len(train[(train.holiday==0) & (train.workingday==1)])]

fig,ax = plt.subplots()
ax.pie(sizes,
      labels = labels)

ax.axis('equal') 
ax.set_title('Day Pie')


plt.show()

In [ ]:
### 0 is Monday and 6 is Sunday
train['dayofweek'] = train.datetime.dt.dayofweek
test['dayofweek'] = test.datetime.dt.dayofweek
train.dayofweek.value_counts() ## -> 요일별로 확인하기

In [ ]:
fig,ax = plt.subplots()
sns.barplot(data = train, x = 'dayofweek', y='count', ax = ax)
ax.set(ylabel='dayofweek', title = 'rent count / dayofweek') 
### 요일도 크게 상관없이 타고 있음

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5,1)
fig.set_size_inches(20,20)
sns.pointplot(data = train, x = 'hour', y = 'count', ax = ax1)
sns.pointplot(data = train, x = 'hour', y = 'count', hue = 'holiday', ax = ax2)
sns.pointplot(data = train, x = 'hour', y = 'count', hue = 'workingday', ax = ax3)
sns.pointplot(data = train, x = 'hour', y = 'count', hue = 'dayofweek', ax = ax4)
sns.pointplot(data = train, x = 'hour', y = 'count', hue = 'season', ax = ax5)

## 평일은 출퇴근시간, 휴일은 오후경에 많이 탄다
## holiday, workingday, dayofweek, season 다 의미 있음 

In [ ]:
train.windspeed.value_counts()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
plt.xticks(rotation = 30, ha = 'right')
sns.countplot(data = train, x = 'windspeed')
ax.set(ylabel = 'windspeed', title = 'windspeed count')

### 

In [ ]:
categorical_feature_names = ["season","holiday","workingday","weather",
                            "dayofweek","month","year","hour"]

for var in categorical_feature_names:
    train[var] = train[var].astype("category")
    test[var] = test[var].astype("category")

### training 

In [ ]:
##관련 있는 컬럼 
## 월, 시간, workingday, holiday, season, 

In [ ]:
feature_names = ['season','weather','temp','atemp','humidity','windspeed',
                'year','hour','dayofweek','holiday','workingday']

feature_names

In [ ]:
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

In [ ]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

In [ ]:
label_name = "count"

y_train = train[label_name]
print(y_train.shape)
y_train.head()

In [ ]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values, convertExp=True):

#     if convertExp:
#         predicted_values = np.exp(predicted_values),
#         actual_values = np.exp(actual_values)
        
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    # 값이 0일 수도 있어서 로그를 취했을 때 마이너스 무한대가 될 수도 있기 때문에 1을 더해 줌
    # 로그를 씌워주는 것은 정규분포로 만들어주기 위해
    log_predict = np.log1p(predicted_values)
    log_actual = np.log1p(actual_values)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

### K_Fold

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
max_depth_list = []

## 높이면 좀 더 좋은 성능 내는데, 시간 오래 걸림

model = RandomForestRegressor(n_estimators=100, 
                             n_jobs = -1,
                             random_state=0)
model

In [ ]:
score = cross_val_score(model,X_train,y_train,cv=k_fold, scoring=rmsle_scorer)

In [ ]:
score = score.mean() # 0에 근접할수록 좋음
print("Score = {0:.5f}".format(score))

In [ ]:
model.fit(X_train,y_train)

In [ ]:
#예측

predictions = model.predict(X_test)
print(predictions.shape)
predictions[0:10]

In [ ]:
submission = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
submission

submission['count'] = predictions

print(submission.shape)
submission.head()

In [ ]:
#submission.to_csv("Score_{0:.5f}_submission.csv".format(score), index=False)
submission.to_csv("submission.csv".format(score), index=False)

In [ ]:
from sklearn.linear_model import Ridge
model_ridge = Ridge(alpha = 2.5)
model_ridge.fit(X_train, y_train)
prd = model_ridge.predict(X_test)

In [ ]:
model_ridge.score(X_train, y_train)

In [ ]:
model_ridge.score(X_test, y_test)

In [ ]:
mean_squared_error(prd, )

In [ ]:
model_ridge

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
def ml_fit(model):
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    score = cross_val_score(model,X_train,y_train,cv=k_fold, scoring=rmsle_scorer)
    print(model)
    print(score)
    return model

In [ ]:
model = ml_fit(RandomForestClassifier(n_estimators = 100))
model = ml_fit(LogisticRegression(solver='lbfgs'))
model = ml_fit(SVC(gamma='scale'))
model = ml_fit(KNeighborsClassifier())
model = ml_fit(GaussianNB())
model = ml_fit(DecisionTreeClassifier())